# Spike Train Calculation

Brief 1-2 sentence description of notebook.

In [1]:
import os
import glob
import git
import sys


In [2]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
# import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import h5py
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter


In [3]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [4]:
git_root

'/blue/npadillacoreano/ryoi360/projects/reward_competition_extention'

In [5]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [6]:
import neuron.spikes

In [7]:
# sns.set('notebook', 'ticks', font_scale=1.2)
mpl.rcParams['figure.figsize'] = [15,6]

## Functions

## Inputs & Data

Explanation of each input and where it comes from.

In [18]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case

LFP_TRACES_DF = pd.read_pickle(os.path.join(git_root, "results/2024_01_30_rce_2nd_cohort/proc/rce2_spectral_granger.pkl"))

# ALL_PHY_DIR = glob.glob("/scratch/back_up/reward_competition_extention/final_proc/phy_curation/*")
ALL_PHY_DIR = glob.glob("/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/*")


OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

SAMPLING_RATE = 20000
SPIKE_WINDOW = 2000

In [19]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,LH_vHPC_granger_all_frequencies_all_windows,vHPC_LH_granger_all_frequencies_all_windows,MD_mPFC_granger_all_frequencies_all_windows,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies
0,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,...,"[[1.216, 0.633, 0.282, 0.613, 0.636, 1.501, 0....","[[0.39, 0.2312, 0.02298, 0.1543, 0.315, 0.7856...","[[1.783, 1.615, 1.699, 0.9365, 1.126, 1.648, 0...","[[1.663, 0.849, 1.167, 0.758, 0.328, 0.7144, 1...","[[2.055, 0.452, 0.3164, 0.5933, 0.8477, 0.6006...","[[0.1825, 0.1715, 0.01872, 0.096, 0.2727, 0.36...","[[0.3892, 0.2598, 0.1339, 0.1823, 0.2996, nan,...","[[0.0007057, 0.2181, 0.2163, 0.5986, 0.6187, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,...,"[[2.848, 1.116, 0.682, 0.198, 0.154, 0.127, 0....","[[1.97, 1.198, 0.10645, 0.2512, 0.3628, 0.595,...","[[0.09326, 0.08734, 0.136, 0.0357, 0.1394, 0.2...","[[1.11, 0.2117, 0.3718, 0.1334, 0.174, 0.7544,...","[[1.734, 1.35, 0.8564, 0.2327, 0.08453, 0.0556...","[[1.0205, 0.565, 0.1328, 0.256, 0.311, 0.3535,...","[[1.432, 2.252, 0.566, 0.302, 0.1381, 0.6523, ...","[[nan, 0.8276, 1.56, 0.672, 0.3506, 0.01371, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
2,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.1,...,"[[0.326, 0.1089, 0.624, 0.2566, 0.06903, 0.112...","[[0.4377, 0.1589, 0.2129, 0.3435, 0.5723, 1.25...","[[0.01956, 0.05756, 0.04272, 0.108, 0.4011, 1....","[[0.1444, 0.5464, 0.3662, 0.3176, 0.2191, 0.08...","[[0.8516, 0.1353, 0.897, 0.2006, 0.02, 0.1517,...","[[0.273, 0.1354, 0.1842, 0.1063, 0.9165, 0.894...","[[0.1587, 0.256, 0.2253, 0.1144, 0.04657, 0.24...","[[0.386, 0.0818, 0.07935, 0.03128, 0.2976, 1.4...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ..."
3,2,20230612_112630_standard_comp_to_training_D1_s...,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948...","[[33019, 33020], [33246, 33251], [33253, 33255...",20230612_112630_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_112630_standard_comp_to_training_D1_s...,1.2,...,"[[0.05106, 0.03592, 0.804, 1.848, 0.3523, 1.55...","[[0.5166, nan, 0.236, 0.238, 0.282, 1.07, 0.03...","[[2.38, nan, 0.556, 0.719, 0.4656, 1.49, 1.627...","[[0.828, 0.4062, 0.2362, 0.0009437, 0.02545, 0...","[[0.

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

In [20]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2"

In [21]:
FULL_LFP_TRACES_PKL = "{}_lfp_traces_sleap_spikes.pkl".format(OUTPUT_PREFIX)

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Reading in Phy

- Reading in a spreadsheet of all the unit classifications
    - They are divided up into good units, multi-units, and noise

In [22]:
ALL_PHY_DIR

['/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/20230620_114347_standard_comp_to_omission_D4_subj_1-2_t3b3L_box_1_merged.rec',
 '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/20230621_111240_standard_comp_to_omission_D5_subj_1-4_t3b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/20230618_100636_standard_comp_to_omission_D2_subj_1_1_t1b2L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec',
 '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl',
 '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/20230619_115321_standard_comp_to_omission_D3_subj_1-4_t3b3L_box2_merged.rec',
 '/blue/npadillacoreano/ryoi360/reward_competition_extention/fina

In [23]:
recording_to_cluster_info = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        recording_to_cluster_info[recording_basename] = pd.read_csv(file_path, sep="\t")
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [24]:
recording_to_cluster_info[list(recording_to_cluster_info.keys())[1]].head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,3,71,22,243,8.729426,1,0,20.0,9.474292,mua,33069,0,5
1,5,4,0,14,9.769127,2,0,40.0,1.879731,mua,6561,0,7
2,6,2,0,11,13.532221,2,0,40.0,11.421351,good,39865,0,8
3,7,58,30,99,9.302568,2,0,40.0,20.851808,good,72781,0,9
4,8,29,12,94,7.241092,3,0,60.0,4.613521,mua,16103,0,10


- Combining all the unit info dataframes and adding the recording name

In [25]:
recording_to_cluster_info_df = pd.concat(recording_to_cluster_info, names=['recording_name']).reset_index(level=1, drop=True).reset_index()


In [26]:
recording_to_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,0,12,9,45,10.498872,0,0.0,0.0,4.379580,mua,14970,0,1.0
1,20230620_114347_standard_comp_to_omission_D4_s...,11,0,0,0,10.311028,5,0.0,100.0,0.230828,mua,789,0,12.0
2,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
3,20230620_114347_standard_comp_to_omission_D4_s...,21,148,57,355,9.774931,11,0.0,220.0,10.004284,mua,34196,0,23.0
4,20230620_114347_standard_comp_to_omission_D4_s...,22,105,26,351,8.435524,12,0.0,240.0,13.887395,mua,47469,0,24.0


- Filtering for the good units

In [27]:
good_unit_cluster_info_df = recording_to_cluster_info_df[recording_to_cluster_info_df["group"] == "good"].reset_index(drop=True)

In [28]:
good_unit_cluster_info_df.head()

,recording_name,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,20230620_114347_standard_comp_to_omission_D4_s...,14,4,0,7,8.454960,8,0.0,160.0,1.908058,good,6522,0,15.0
1,20230620_114347_standard_comp_to_omission_D4_s...,28,2,1,7,8.708598,14,0.0,280.0,1.436163,good,4909,0,30.0
2,20230620_114347_standard_comp_to_omission_D4_s...,41,2,1,3,10.409801,19,0.0,380.0,1.583319,good,5412,0,43.0
3,20230620_114347_standard_comp_to_omission_D4_s...,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0
4,20230620_114347_standard_comp_to_omission_D4_s...,43,1,0,4,7.047093,26,0.0,520.0,1.489701,good,5092,0,45.0


In [29]:
recording_to_good_unit_ids = good_unit_cluster_info_df.groupby('recording_name')['cluster_id'].apply(list).to_dict()

- A list of all the unit IDs that each spike came from in order
    - First item is first spike, second item is second spike, etc.

In [30]:
recording_to_spike_clusters = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        recording_to_spike_clusters[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_clusters.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_clusters.npy'


In [31]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]]

array([ 42,  42, 207, ...,  22, 229,  68], dtype=int32)

In [32]:
recording_to_spike_clusters[list(recording_to_spike_clusters.keys())[0]].shape

(903326,)

- The times that all the spikes happened

In [33]:
recording_to_spike_times = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        file_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        recording_to_spike_times[recording_basename] = np.load(file_path)
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/spike_times.npy'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/spike_times.npy'


In [34]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]]

array([[     120],
       [     262],
       [     439],
       ...,
       [68362487],
       [68362524],
       [68362610]])

In [35]:
recording_to_spike_times[list(recording_to_spike_times.keys())[0]].shape

(903326, 1)

### Combining everything into a dataframe

In [36]:
recording_to_spike_df = {}
for recording_dir in ALL_PHY_DIR:
    try:
        recording_basename = os.path.basename(recording_dir).strip(".rec")
        cluster_info_path = os.path.join(recording_dir, "phy", "cluster_info.tsv")
        cluster_info_df = pd.read_csv(cluster_info_path, sep="\t")

        spike_clusters_path = os.path.join(recording_dir, "phy", "spike_clusters.npy")
        spike_clusters = np.load(spike_clusters_path)
        
        spike_times_path = os.path.join(recording_dir, "phy", "spike_times.npy")
        spike_times = np.load(spike_times_path)

        spike_df = pd.DataFrame({'spike_clusters': spike_clusters, 'spike_times': spike_times.T[0]})

        merged_df = spike_df.merge(cluster_info_df, left_on='spike_clusters', right_on='cluster_id', how="left")
        merged_df["recording_name"] = recording_basename

        merged_df["timestamp_isi"] = merged_df.groupby('spike_clusters')["spike_times"].diff()
        merged_df["current_isi"] = merged_df["timestamp_isi"] / SAMPLING_RATE
        
        if not merged_df.empty:
            recording_to_spike_df[recording_basename] = merged_df
       
    except Exception as e:
        print(e)

[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_trodes_metadata.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_video_frames.pkl/phy/cluster_info.tsv'
[Errno 20] Not a directory: '/blue/npadillacoreano/ryoi360/reward_competition_extention/final_proc/phy_curation/rce_pilot_2_full_lfp_traces_and_sleap_poses.pkl/phy/cluster_info.tsv'


In [37]:
cluster_info_df.head()

,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id
0,2,0,0,0,24.692951,1,0.0,20.0,1.070175,good,3658,0,3.0
1,3,235,106,472,10.347874,1,0.0,20.0,14.493880,mua,49542,0,4.0
2,4,64,25,149,11.953332,4,0.0,80.0,27.163371,mua,92848,0,5.0
3,10,69,29,194,10.297276,18,0.0,360.0,6.455279,mua,22065,0,12.0
4,11,619,261,1295,9.638424,19,0.0,380.0,18.195901,mua,62196,0,13.0


In [38]:
spike_times

array([[     160],
       [     483],
       [     503],
       ...,
       [68361973],
       [68362204],
       [68362381]])

In [39]:
spike_clusters

array([ 4,  4, 11, ...,  4,  4, 87], dtype=int32)

- Combining the spike time df for all recordings

In [40]:
all_spike_time_df = pd.concat(recording_to_spike_df.values())

In [41]:
all_spike_time_df = all_spike_time_df[all_spike_time_df["group"] == "good"].reset_index(drop=True)

In [42]:
all_spike_time_df.head()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
0,42,120,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
1,42,262,42,6,1,8,11.904501,20,0.0,400.0,12.966718,good,44322,0,44.0,20230620_114347_standard_comp_to_omission_D4_s...,142.0,0.00710
2,207,439,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN
3,207,632,207,1,0,5,16.813625,17,0.0,340.0,4.308197,good,14726,0,36.0,20230620_114347_standard_comp_to_omission_D4_s...,193.0,0.00965
4,262,723,262,0,0,0,20.012720,19,0.0,380.0,2.037368,good,6964,0,39.0,20230620_114347_standard_comp_to_omission_D4_s...,NaN,NaN


In [43]:
all_spike_time_df.tail()

,spike_clusters,spike_times,cluster_id,1_5_ms_isi,1ms_isi,2ms_isi,amp,ch,channel_group,depth,fr,group,n_spikes,sh,si_unit_id,recording_name,timestamp_isi,current_isi
2017369,64,68356117,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,392.0,0.0196
2017370,89,68356258,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,3360.0,0.1680
2017371,64,68357205,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1088.0,0.0544
2017372,89,68357578,89,11,11,13,17.238409,26,0.0,520.0,15.528948,good,53080,0,21.0,20230620_114347_standard_comp_to_omission_D4_s...,1320.0,0.0660
2017373,64,68358945,64,4,1,12,10.794415,14,0.0,280.0,7.735803,good,26442,0,9.0,20230620_114347_standard_comp_to_omission_D4_s...,1740.0,0.0870


## Grouping all the neurons by recording

In [44]:
# Grouping all spike times by neuron and recording

grouped_df = all_spike_time_df.groupby(['spike_clusters', 'recording_name'])["spike_times"].apply(lambda x: sorted(list(x))).reset_index()
grouped_df = grouped_df.sort_values(by=['recording_name', 'spike_clusters']).reset_index(drop=True)
grouped_df = grouped_df.groupby('recording_name').agg({'spike_clusters': lambda x: list(x), 'spike_times': lambda x: list(x)}).reset_index()

In [45]:
grouped_df

,recording_name,spike_clusters,spike_times
0,20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ..."
1,20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2..."
2,20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120..."
3,20230618_100636_standard_comp_to_omission_D2_s...,"[9, 19, 65, 83, 87, 93, 99, 103, 105, 106, 118...","[[458431, 506645, 509846, 545481, 571000, 6011..."
4,20230619_115321_standard_comp_to_omission_D3_s...,"[11, 44, 132, 134, 186, 187, 227, 250, 252, 25...","[[17747, 20655, 24011, 34755, 67305, 110220, 1..."
5,20230620_114347_standard_comp_to_omission_D4_s...,"[2, 22, 64, 89, 98, 100, 119, 121, 122, 126]","[[264055, 325873, 327931, 330996, 334463, 3590..."
6,20230620_114347_standard_comp_to_omission_D4_s...,"[14, 28, 41, 42, 43, 45, 48, 53, 58, 68, 124, ...","[[5884, 7172, 11405, 19311, 19371, 23729, 2627..."
7,20230621_111240_standard_comp_to_omission_D5_s...,"[6, 7, 13, 15, 16, 17, 22, 24, 26, 36, 37, 46,...","[[1891, 5872, 8366, 16312, 17233, 24302, 24337..."


In [46]:
LFP_TRACES_DF = pd.merge(LFP_TRACES_DF, grouped_df, left_on='recording', right_on="recording_name", how='inner')

## Calculating the firing rates

In [47]:
LFP_TRACES_DF.columns

Index(['cohort', 'session_dir', 'original_file', 'tone_frames',
       'box_1_port_entry_frames', 'box_2_port_entry_frames', 'video_name',
       'session_path', 'recording', 'current_subject', 'subject',
       'all_subjects', 'first_timestamp', 'last_timestamp', 'video_timestamps',
       'tone_timestamps', 'box_1_port_entry_timestamps',
       'box_2_port_entry_timestamps', 'lfp_timestamps', 'mPFC_lfp_trace',
       'MD_lfp_trace', 'LH_lfp_trace', 'BLA_lfp_trace', 'vHPC_lfp_trace',
       'mPFC_power_all_frequencies_all_windows',
       'MD_power_all_frequencies_all_windows',
       'LH_power_all_frequencies_all_windows',
       'BLA_power_all_frequencies_all_windows',
       'vHPC_power_all_frequencies_all_windows', 'power_timestamps',
       'power_calculation_frequencies',
       'BLA_LH_coherence_all_frequencies_all_windows',
       'BLA_MD_coherence_all_frequencies_all_windows',
       'BLA_mPFC_coherence_all_frequencies_all_windows',
       'BLA_vHPC_coherence_all_frequencies_

In [48]:
LFP_TRACES_DF.head()

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,...,mPFC_MD_granger_all_frequencies_all_windows,MD_vHPC_granger_all_frequencies_all_windows,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies,recording_name,spike_clusters,spike_times
0,2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1,...,"[[0.856, 0.529, 0.01147, 0.02261, 0.0289, 0.08...","[[0.4624, 0.011536, 0.868, 0.2058, 1.5625, 0.6...","[[0.508, 0.345, 0.004494, 0.1903, 0.0349, 0.24...","[[0.03137, 0.00545, 0.9624, 0.412, 0.533, 0.21...","[[3.105, 1.563, 0.0859, 0.1681, 0.2595, 0.3398...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ..."
1,2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2,...,"[[0.0986, 0.6724, 0.839, 0.2634, 1.138, 3.412,...","[[0.6396, 0.2905, 0.0549, 0.0666, 0.4778, 1.30...","[[1.719, 0.1829, 0.1862, 0.01852, 0.01707, 0.0...","[[0.1705, 0.44, 0.01059, 0.07196, 0.1414, 0.36...","[[nan, 0.2847, 0.397, 0.3748, 0.382, 0.0392, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2..."
2,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1,...,"[[0.11444, 0.137, 0.2966, 0.1887, 0.3015, 0.15...","[[0.389, 0.01688, 0.10986, 0.0417, 0.2942, 0.5...","[[0.2257, 0.0779, 0.599, 0.1936, 0.1547, 0.048...","[[0.178, 0.04852, 0.02904, 0.04523, 0.468, 0.3...","[[0.1025, 0.7397, 1.007, 0.5337, 0.135, 0.0441...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120..."
3,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4,...,"[[0.9688, 0.9434, 1.153, 0.9565, 0.7573, 0.609...","[[1.176, 5.562, 0.010704, 0.2837, 0.1097, 0.38...","[[0.08704, 0.06995, 0.11365, 1.118, 1.183, 0.7...","[[0.4653, nan, 0.5586, 0.3257, 0.479, 0.5654, ...","[[0.3765, 0.1475, 0.2418, 1.542, 0.697, 0.3938...","[10000, 20000, 30000, 40000, 50000, 60000, 700..

In [52]:
LFP_TRACES_DF["neuron_average_fr"] = LFP_TRACES_DF.apply(lambda x: np.array([neuron.spikes.calculate_rolling_avg_firing_rate(np.array(times), stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[0] for times in x["spike_times"]]), axis=1)

In [54]:
LFP_TRACES_DF["neuron_average_timestamps"] = LFP_TRACES_DF.apply(lambda x: neuron.spikes.calculate_rolling_avg_firing_rate(x["spike_times"][0], stop_time=x["last_timestamp"] - x["first_timestamp"], window_size=SPIKE_WINDOW, slide=SPIKE_WINDOW)[1], axis=1)

In [55]:
LFP_TRACES_DF["neuron_average_fr"] = LFP_TRACES_DF.apply(lambda x: x["neuron_average_fr"] * SPIKE_WINDOW, axis=1)

In [56]:
LFP_TRACES_DF.iloc[:5,:10]

,cohort,session_dir,original_file,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject
0,2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.1
1,2,20230617_115521_standard_comp_to_omission_D1_s...,20230617_115521_standard_comp_to_omission_D1_s...,"[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0...","[[0, 0], [0, 0], [0, 0], [0, 5], [12, 13], [13...",20230617_115521_standard_comp_to_omission_D1_s...,/scratch/back_up/reward_competition_extention/...,20230617_115521_standard_comp_to_omission_D1_s...,1.2
2,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.1
3,2,20230618_100636_standard_comp_to_omission_D2_s...,20230618_100636_standard_comp_to_omission_D2_s...,"[[1098, 1297], [3492, 3693], [5788, 5988], [75...","[[788, 835], [838, 839], [839, 871], [871, 900...","[[33026, 33037], [33038, 33042], [33044, 33049...",20230618_100636_standard_comp_to_omission_D2_s...,/scratch/back_up/reward_competition_extention/...,20230618_100636_standard_comp_to_omission_D2_s...,1.4
4,2,20230619_115321_standard_comp_to_omission_D3_s...,20230619_115321_standard_comp_to_omission_D3_s...,"[[1275, 1475], [3571, 3770], [5368, 5567], [64...","[[593, 603], [634, 640], [840, 897], [907, 983...","[[31077, 31084], [31087, 31116], [31119, 31129...",20230619_115321_standard_comp_to_omission_D3_s...,/scratch/back_up/reward_competition_extention/...,20230619_115321_standard_comp_to_omission_D3_s...,1.4


In [57]:
LFP_TRACES_DF.iloc[:5,10:20]

,subject,all_subjects,first_timestamp,last_timestamp,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace
0,1.1,"[1.1, 1.2]",3478533,83177118,"[40669726, 40671112, 40671112, 40672498, 40673...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[[25030476, 31380073], [39607470, 39621277], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.07367, 0.1105, 0.19, 0.2365, 0.0717, -0.160..."
1,1.2,"[1.1, 1.2]",3478533,83177118,"[40669726, 40671112, 40671112, 40672498, 40673...","[[1030879, 1230878], [3430878, 3630878], [5730...","[[102478, 104881], [110481, 147879], [193083, ...","[[25030476, 31380073], [39607470, 39621277], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.9487, 1.179, 1.587, 1.677, 1.536, 1.568, 1...."
2,1.1,"[1.1, 1.4]",835680,66439621,"[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[[33057437, 33069437], [33070037, 33075039], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.3154, -0.2812, -0.2698, -0.3381, -0.2261, ..."
3,1.4,"[1.1, 1.4]",835680,69429292,"[-2, 1384, 2770, 2770, 4155, 5541, 6927, 6927,...","[[1099250, 1299250], [3499256, 3699252], [5799...","[[788852, 836852], [838652, 840050], [840652, ...","[[33057437, 33069437], [33070037, 33075039], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[0.0823, 0.0658, -0.1316, -0.7896, -1.333, -1...."
4,1.4,"[1.2, 1.4]",1642256,42829633,"[-2, 1384, 2770, 2770, 4156, 5541, 5541, 6927,...","[[1277472, 1477469], [3577468, 3777470], [5377...","[[594072, 603070], [635272, 640869], [842269, ...","[[31136254, 31142253], [31144653, 31174253], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[-0.6226, -0.82, -0.7827, -0.8125, -1.267, -1...."


In [58]:
LFP_TRACES_DF.iloc[:5,20:30]

,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace,mPFC_power_all_frequencies_all_windows,MD_power_all_frequencies_all_windows,LH_power_all_frequencies_all_windows,BLA_power_all_frequencies_all_windows,vHPC_power_all_frequencies_all_windows,power_timestamps
0,"[0.1039, -0.011925, 0.02043, 0.1362, 0.011925,...","[0.004, -0.1181, -0.05203, 0.1702, 0.1521, 0.0...","[0.2854, 0.8105, 1.242, 1.026, 0.6797, 0.294, ...","[-0.2625, -0.1997, -0.01572, 0.1509, 0.02202, ...","[[0.0237, 0.01284, 0.02682, 0.04108, 0.05316, ...","[[0.002655, 0.01074, 0.01842, 0.02864, 0.0338,...","[[0.00533, 0.01519, 0.0261, 0.03506, 0.0361, 0...","[[0.03333, 0.01772, 0.02557, 0.019, 0.02469, 0...","[[0.00755, 0.005825, 0.0322, 0.04767, 0.0523, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700..."
1,"[0.8105, 0.909, 0.8447, 0.6675, 0.914, 1.372, ...","[1.025, 1.567, 1.864, 1.6455, 1.957, 2.387, 2....","[0.577, 1.227, 1.856, 1.917, 2.18, 2.404, 2.46...","[-0.5967, -0.5806, -0.5005, -0.3042, 0.0, 0.18...","[[0.03217, 0.03186, 0.07117, 0.0814, 0.08716, ...","[[0.08264, 0.06064, 0.0753, 0.0392, 0.03656, 0...","[[0.01704, 0.0323, 0.04984, 0.05484, 0.05585, ...","[[0.004166, 0.0266, 0.0355, 0.0466, 0.05707, 0...","[[0.001713, 0.01749, 0.01785, 0.01508, 0.01006...","[10000, 20000, 30000, 40000, 50000, 60000, 700..."
2,"[0.003227, -0.06616, -0.0742, 0.0565, 0.1743, ...","[0.3372, 0.4973, 0.524, 0.3906, 0.4954, 0.56, ...","[0.1832, 0.4846, 0.766, 0.8545, 0.961, 0.7627,...","[0.9727, 1.524, 2.242, 2.426, 2.252, 1.847, 1....","[[0.015305, 0.0223, 0.03632, 0.02211, 0.02168,...","[[0.00202, 0.00401, 0.012985, 0.011696, 0.0151...","[[0.006813, 0.00756, 0.01591, 0.01199, 0.01014...","[[0.01706, 0.02808, 0.02686, 0.01572, 0.02292,...","[[0.01005, 0.01069, 0.01654, 0.0264, 0.02428, ...","[10000, 20000, 30000, 40000, 50000, 60000, 700..."
3,"[-0.4082, -0.651, -0.852, -0.8345, -0.8223, -0...","[-0.846, -1.169, -1.349, -1.064, -0.8716, -0.9...","[-0.1432, -0.525, -1.298, -1.044, -0.3499, -1....","[-0.755, -1.41, -2.273, -2.297, -2.133, -2.557...","[[0.02173, 0.01424, 0.01416, 0.00531, 0.03857,...","[[0.004864, 0.004448, 0.0083, 0.009125, 0.0586...","[[0.002157, 0.002275, 0.00783, 0.009575, 0.071...","[[0.0183, 0.00981, 0.02113, 0.01231, 0.0809, 0...","[[0.005535, 0.00698, 0.03174, 0.03128, 0.1538,...","[10000, 20000, 30000, 40000, 50000, 60000, 700..."
4,"[-1.124, -1.211, -1.179, -1.127, -1.054, -0.65...","[-0.6846, -0.721, -0.5845, -0.561, -0.6445, -0...","[-0.844, -1.334, -1.739, -1.722, -1.799, -1.61...","[-0.146, -0.2466, -0.3926, -0.6367, -0.9463, -...","[[0.01159, 0.006756, 0.01059, 0.0102, 0.01204,...","[[0.00909, 0.0406, 0.0516, 0.07306, 0.05405, 0...","[[0.018, 0.04575, 0.05255, 0.0682, 0.03833, 0....","[[0.03008, 0.03418, 0.03455, 0.0275, 0.02121, ...","[[0.0428, 0.10046, 0.1033, 0.11163, 0.07025, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700..."


In [59]:
LFP_TRACES_DF.iloc[:5, 30:]

,power_calculation_frequencies,BLA_LH_coherence_all_frequencies_all_windows,BLA_MD_coherence_all_frequencies_all_windows,BLA_mPFC_coherence_all_frequencies_all_windows,BLA_vHPC_coherence_all_frequencies_all_windows,LH_MD_coherence_all_frequencies_all_windows,LH_mPFC_coherence_all_frequencies_all_windows,LH_vHPC_coherence_all_frequencies_all_windows,MD_mPFC_coherence_all_frequencies_all_windows,MD_vHPC_coherence_all_frequencies_all_windows,...,vHPC_MD_granger_all_frequencies_all_windows,mPFC_vHPC_granger_all_frequencies_all_windows,vHPC_mPFC_granger_all_frequencies_all_windows,granger_timestamps,granger_calculation_frequencies,recording_name,spike_clusters,spike_times,neuron_average_fr,neuron_average_timestamps
0,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[[0.116, 0.34, 0.4612, 0.836, 0.853, 0.5596, 0...","[[0.4922, 0.2343, 0.4539, 0.7573, 0.831, 0.608...","[[0.627, 0.7295, 0.816, 0.951, 0.9697, 0.8765,...","[[0.625, 0.7666, 0.7227, 0.9277, 0.816, 0.6367...","[[0.8076, 0.969, 0.9854, 0.9785, 0.949, 0.91, ...","[[0.7026, 0.548, 0.8, 0.8486, 0.832, 0.581, 0....","[[0.7124, 0.767, 0.734, 0.8525, 0.99, 0.983, 0...","[[0.9395, 0.5327, 0.776, 0.81, 0.848, 0.609, 0...","[[0.959, 0.6475, 0.695, 0.807, 0.9585, 0.888, ...",...,"[[0.508, 0.345, 0.004494, 0.1903, 0.0349, 0.24...","[[0.03137, 0.00545, 0.9624, 0.412, 0.533, 0.21...","[[3.105, 1.563, 0.0859, 0.1681, 0.2595, 0.3398...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[3, 4, 5, 6, 17, 28, 29, 39, 48, 52, 53, 54, 5...","[[673223, 2367433, 2569212, 3969647, 3973243, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
1,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[[0.727, 0.4612, 0.642, 0.9824, 0.9873, 0.984,...","[[0.8467, 0.1937, 0.177, 0.961, 0.9243, 0.8643...","[[0.5806, 0.3042, 0.5464, 0.9565, 0.872, 0.777...","[[0.773, 0.975, 0.10223, 0.147, 0.561, 0.772, ...","[[0.975, 0.7227, 0.7573, 0.9707, 0.9634, 0.931...","[[0.9614, 0.875, 0.908, 0.9316, 0.902, 0.8696,...","[[0.8076, 0.4446, 0.06335, 0.1443, 0.501, 0.67...","[[0.8833, 0.9585, 0.7524, 0.8984, 0.97, 0.9766...","[[0.81, 0.10754, 0.1426, 0.2534, 0.3474, 0.51,...",...,"[[1.719, 0.1829, 0.1862, 0.01852, 0.01707, 0.0...","[[0.1705, 0.44, 0.01059, 0.07196, 0.1414, 0.36...","[[nan, 0.2847, 0.397, 0.3748, 0.382, 0.0392, 0...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230617_115521_standard_comp_to_omission_D1_s...,"[16, 17, 20, 25, 26, 34, 35, 39, 40, 42, 43, 4...","[[958, 5233, 6331, 7475, 8599, 10550, 23459, 2...","[[1.0, 0.0, 1.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
2,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[[0.539, 0.8413, 0.714, 0.6323, 0.6743, 0.8394...","[[0.0107, 0.5386, 0.4678, 0.5063, 0.715, 0.814...","[[0.818, 0.9517, 0.8213, 0.7104, 0.537, 0.1165...","[[0.8555, 0.9033, 0.846, 0.6855, 0.4634, 0.003...","[[0.5137, 0.8047, 0.8423, 0.8394, 0.783, 0.743...","[[0.1504, 0.6855, 0.815, 0.9004, 0.836, 0.3022...","[[0.86, 0.8716, 0.9053, 0.695, 0.647, 0.00902,...","[[0.06226, 0.4448, 0.678, 0.85, 0.6284, 0.2715...","[[0.1669, 0.482, 0.66, 0.351, 0.2292, 0.0636, ...",...,"[[0.2257, 0.0779, 0.599, 0.1936, 0.1547, 0.048...","[[0.178, 0.04852, 0.02904, 0.04523, 0.468, 0.3...","[[0.1025, 0.7397, 1.007, 0.5337, 0.135, 0.0441...","[10000, 20000, 30000, 40000, 50000, 60000, 700...","[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...",20230618_100636_standard_comp_to_omission_D2_s...,"[7, 11, 12, 48, 50, 51, 61, 64, 76, 84, 90, 98...","[[506, 1419, 2358, 5104, 6785, 7620, 9801, 120...","[[2.0, 1.0, 1.0, 2.0, 1.0, 0.0, 3.0, 0.0, 1.0,...","[0, 2000, 4000, 6000, 8000, 10000, 12000, 1400..."
3,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[[0.811, 0.2367, 0.7607, 0.7485, 0.913, 0.8623...","[[0.973, 0.3767, 0.662, 0.607, 0.9136, 0.

In [51]:
LFP_TRACES_DF["neuron_average_timestamps"].iloc[0].shape

(19787,)

In [52]:
LFP_TRACES_DF["neuron_average_fr"].iloc[0].shape

(18, 19787)

In [49]:
np.max(LFP_TRACES_DF["neuron_average_fr"].iloc[0])

16.0

In [46]:
LFP_TRACES_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [47]:
raise ValueError("")

SyntaxError: invalid syntax (237021957.py, line 1)